In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
import os
import time 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import zero_one_loss
import pickle
import random

In [3]:
#%%划分训练集和测试集
#读取数据集
train=pd.read_csv('data/train1.csv')
test=pd.read_csv('data/test1.csv')

In [4]:
features=list(train.columns)
features.remove('flag')
X_train, y_train=train[features],train['flag']
X_test = test[features]

# 交叉验证

In [5]:
#%%预测框架
prob_oof = np.zeros((X_train.shape[0], ))
result = np.zeros((X_train.shape[0], ))
test_pred_prob = np.zeros((X_test.shape[0], ))
test_result=[]
#prediction=pd.DataFrame(y_test)
prediction=pd.DataFrame(test['id'])

## gbdt

In [6]:
#%%模型求解
start = time.clock()

##k折交叉验证
folds = KFold(n_splits=10, shuffle=True, random_state=2019)

##训练和测试
feature_importance_df = pd.DataFrame()
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train,y_train)):
    print("fold {}".format(fold_ + 1))
    trn_data, trn_label = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_data, val_label = X_train.iloc[val_idx], y_train.iloc[val_idx]

    clf = GradientBoostingClassifier(learning_rate = 0.01, max_features = 0.8, n_estimators = 1000 , max_depth = 5).fit(trn_data, trn_label)
    prob_oof[val_idx] = clf.predict_proba(val_data)[:,1]
    
    threshold = 0.5
    for i in val_idx:
        result[i] = 1 if prob_oof[i] > threshold else 0
    test_pred_prob += clf.predict_proba(X_test)[:,1] / folds.n_splits

print("CV score(ACC): {:<8.5f}".format(accuracy_score(y_train, result)))
print("CV score(AUC): {:<8.5f}".format(roc_auc_score(y_train, prob_oof)))
print ('CV score(F1): %.4f' % f1_score(y_train, result,average='weighted'))


##计算运行时间
elapsed = (time.clock() - start)
print("Time used:",elapsed)

fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
fold 10
CV score(ACC): 0.87191 
CV score(AUC): 0.92460 
CV score(F1): 0.8712
Time used: 5826.083414


In [7]:
minmin = min(prob_oof)
maxmax = max(prob_oof)
vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
prob_oof=vfunc(prob_oof)

minmin = min(test_pred_prob)
maxmax = max(test_pred_prob)
vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
test_pred_prob=vfunc(test_pred_prob)

In [14]:
##测试
for pred in test_pred_prob:
    s = 1 if pred > threshold else 0
    test_result.append(s)
minmin = min(test_pred_prob)
maxmax = max(test_pred_prob)
vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
prediction['pred']=vfunc(test_pred_prob)
#prediction['pred']=test_result 
#print ('ACC: %.4f' % accuracy_score(y_test,test_result))
#print("AUC: {:<8.5f}".format(roc_auc_score(y_test, test_pred_prob)))
#print ('F1: %.4f' % f1_score(y_test,test_result,average='weighted'))

In [8]:
# 获取时间
time_str = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
# 保存模型
out_dir = "result/gbdt_result/{}/".format(time_str)
os.makedirs(out_dir)


#保存结果
prediction.to_csv(out_dir + "prediction.csv",index=False)
np.save(out_dir + "prediction_gbdt.npy",test_pred_prob)
np.save(out_dir + "oof_gbdt.npy",prob_oof)

# gbdt model2

In [9]:
#%%预测框架
prob_oof = np.zeros((X_train.shape[0], ))
result = np.zeros((X_train.shape[0], ))
test_pred_prob = np.zeros((X_test.shape[0], ))
test_result=[]
#prediction=pd.DataFrame(y_test)
prediction=pd.DataFrame(test['id'])

In [10]:
#%%模型求解
start = time.clock()

##k折交叉验证
folds = KFold(n_splits=10, shuffle=True, random_state=2019)

##训练和测试
feature_importance_df = pd.DataFrame()
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train,y_train)):
    print("fold {}".format(fold_ + 1))
    trn_data, trn_label = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_data, val_label = X_train.iloc[val_idx], y_train.iloc[val_idx]

    clf = GradientBoostingClassifier(learning_rate = 0.01, max_features = 0.7, n_estimators = 700 , max_depth = 5).fit(trn_data, trn_label)
    prob_oof[val_idx] = clf.predict_proba(val_data)[:,1]
    
    threshold = 0.5
    for i in val_idx:
        result[i] = 1 if prob_oof[i] > threshold else 0
    test_pred_prob += clf.predict_proba(X_test)[:,1] / folds.n_splits

print("CV score(ACC): {:<8.5f}".format(accuracy_score(y_train, result)))
print("CV score(AUC): {:<8.5f}".format(roc_auc_score(y_train, prob_oof)))
print ('CV score(F1): %.4f' % f1_score(y_train, result,average='weighted'))


##计算运行时间
elapsed = (time.clock() - start)
print("Time used:",elapsed)

fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
fold 10
CV score(ACC): 0.87036 
CV score(AUC): 0.92369 
CV score(F1): 0.8696
Time used: 3842.857196


In [11]:
# 获取时间
time_str = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
# 保存模型
out_dir = "result/gbdt2_result/{}/".format(time_str)
os.makedirs(out_dir)


#保存结果
prediction.to_csv(out_dir + "prediction.csv",index=False)
np.save(out_dir + "prediction_gbdt.npy",test_pred_prob)
np.save(out_dir + "oof_gbdt.npy",prob_oof)

## randomForest

In [11]:
#%%预测框架
prob_oof = np.zeros((X_train.shape[0], ))
result = np.zeros((X_train.shape[0], ))
test_pred_prob = np.zeros((X_test.shape[0], ))
test_result=[]
#prediction=pd.DataFrame(y_test)
prediction=pd.DataFrame(test['id'])

In [ ]:
#%%模型求解
start = time.clock()

##k折交叉验证
folds = KFold(n_splits=5, shuffle=True, random_state=2019)

##训练和测试
feature_importance_df = pd.DataFrame()
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train,y_train)):
    print("fold {}".format(fold_ + 1))
    trn_data, trn_label = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_data, val_label = X_train.iloc[val_idx], y_train.iloc[val_idx]

    clf = ExtraTreesClassifier(max_features = 'log2', n_estimators = 1000 , n_jobs = -1).fit(trn_data, trn_label)
    prob_oof[val_idx] = clf.predict_proba(val_data)[:,1]
    
    threshold = 0.5
    for i in val_idx:
        result[i] = 1 if prob_oof[i] > threshold else 0
    test_pred_prob += clf.predict_proba(X_test)[:,1] / folds.n_splits

print("CV score(ACC): {:<8.5f}".format(accuracy_score(y_train, result)))
print("CV score(AUC): {:<8.5f}".format(roc_auc_score(y_train, prob_oof)))
print ('CV score(F1): %.4f' % f1_score(y_train, result,average='weighted'))


##计算运行时间
elapsed = (time.clock() - start)
print("Time used:",elapsed)

fold 1
fold 2
fold 3


In [ ]:
minmin = min(prob_oof)
maxmax = max(prob_oof)
vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
prob_oof=vfunc(prob_oof)

minmin = min(test_pred_prob)
maxmax = max(test_pred_prob)
vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
test_pred_prob=vfunc(test_pred_prob)

In [19]:
##测试
for pred in test_pred_prob:
    s = 1 if pred > threshold else 0
    test_result.append(s)
minmin = min(test_pred_prob)
maxmax = max(test_pred_prob)
vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
prediction['pred']=vfunc(test_pred_prob)
#prediction['pred']=test_result 
#print ('ACC: %.4f' % accuracy_score(y_test,test_result))
#print("AUC: {:<8.5f}".format(roc_auc_score(y_test, test_pred_prob)))
#print ('F1: %.4f' % f1_score(y_test,test_result,average='weighted'))

In [20]:
# 获取时间
time_str = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
# 保存模型
out_dir = "result/rf_result/{}/".format(time_str)
os.makedirs(out_dir)


#保存结果
prediction.to_csv(out_dir + "prediction.csv",index=False)
np.save(out_dir + "prediction_rf.npy",test_pred_prob)
np.save(out_dir + "oof_rf.npy",prob_oof)

In [22]:
prediction.to_csv('predict_01.txt', sep = '\t', index = 0, header = 0)